In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Activity 2: Numerical Methods

## Preliminaries

This unit is all about computing the solutions of differential equations *numerically*. We'll need to introduce a few programming concepts to make this work.

1. **Loops**: A loop is a way to repeat the same code many times. This is important when solving differential equations numerically because the process is inherantly iterative. We break the problem down into tiny steps and we use the differential equation to tell us how each step should progress. 
2. **Functions**: It's useful to separate out bits of the solution process into independent pieces. One nice way to do that is to make each part a *function* that just knows how to handle that one part.
3. **Lists**: We can use lists to save the results of the calculation as the process continues.
4. **Arrays**: An array is a kind of high-performance list of all the same kinds of values.

Let's go through these one-by-one in detail.

## Loops

In general a loop is a structure that lets you repeat code many times. There are two basic types: `for` loops and `while` loops.

### For Loops

Here's a simple `for` loop in python:

In [3]:
for k in [0,1,2,3,4]:
    print("k=",k,"k**2=",k**2)

k= 0 k**2= 0
k= 1 k**2= 1
k= 2 k**2= 4
k= 3 k**2= 9
k= 4 k**2= 16


Note that `k**2` is how you spell exponentiation (i.e., $k^2$) in python.

A `for` loop always starts like `for foo in bar:`, where `foo` is the "loop control variable" and `bar` is something that can be iterated over, often a list or an array. There is a handy utility called `range` that can be used to generate an iterator on the fly.

In [6]:
for k in range(5):
    print("k=",k,"k**2=",k**2)

k= 0 k**2= 0
k= 1 k**2= 1
k= 2 k**2= 4
k= 3 k**2= 9
k= 4 k**2= 16


Note that, by default, range starts with zero. We'll see how this is useful when we get to list indexing.

### While Loops

A while loop requires a condition that can, in principle, be either true or false. Here is an example of a while loop:

In [7]:
k = 0
while k < 5:
    print("k=",k,"k**2=",k**2)
    k = k + 1

k= 0 k**2= 0
k= 1 k**2= 1
k= 2 k**2= 4
k= 3 k**2= 9
k= 4 k**2= 16


Note that this loop does exactly the same thing as the for loop, but takes more typing. You have to "set up" the variables in the condition beforehand. You also need to manually adjust the variables used in the condition each time through the loop. Sometimes, this is a natural side effect of the calculation, so it's no *extra* work. The good thing about a `while` loop is that it's very general. There's nothing a `for` loop can do that a `while` loop can't. However, it takes more bookkeeping and typing. This is the price we pay for generality. In the context of differential equations, we'll more often be using the `while` loop, but not always.

## Functions

A *function* is a packaged bit of code that performs a specific task. Sometimes it returns a value, like a mathematical function (e.g., `np.sin(x)`), but sometimes it just does something but doesn't return anything (e.g., `print("hello")`). You can define your own functions using the `def` keyword in python:

In [8]:
def add3(x):
    return x+3

This example, `add3`, is a simple function that accepts one argument `x`, and returns the value of `x` plus 3. Let's test it!

In [17]:
y = add3(5)
print("y =",y)

y = 8


Simple, right! One thing that sometimes confuses programmers who are first learning about functions is the nature of the arguments. The variables used as arguments in a function are just *placeholders*. They do not refer to variables of the same name in other parts of the program. Let's see how that works.

In [16]:
x = 9

def add3(x):
    return x+3

y = add3(5)

print("x =", x, ", y =",y)

x = 9 , y = 8


Notice that even though `x` was equal to 5 in the function, the value of `x` outside the function was always 9. The arguments to a function, and the variables *assigned* in a function are called *local* variables. They don't refer to variables outside the function. If you don't *assign* a variable in a function, and it's not an argument, python will look outside the function to resolve the value of the varible. That's what's confusing!

In [18]:
x = 9
y = 4

def add_y(x):
    return x + y

z = add_y(5)

print("x =", x, ", y =", y, ", z =",z)

x = 9 , y = 4 , z = 9


What's the main point? You can define a function that accepts values, calculates things, and returns values. It's important to keep track of which variables are *local* and which are not. When possible all the information a function needs to do it's job should be passed in as arguments to the function.

## Lists

A list is just a collection of things. You can create a list very easily with brackets in python: `myList = []`. This creates an empty list called `myList`. You can also create a list with things in it:

In [19]:
myList = [1,2,"hello",3.4,[5,6,7]]

for thing in myList:
    print("This thing is:",thing)



This thing is: 1
This thing is: 2
This thing is: hello
This thing is: 3.4
This thing is: [5, 6, 7]


Note how easy it is to use a `for` loop to iterate through the contents of a list! Lists and `for` loops work great together.

Probably the most important feature of a python list is the ability to add things to it dynamically during program execution:

In [21]:
myList = []

for k in range(5):
    myList.append(k**2)

print("my list is now:", myList)

my list is now: [0, 1, 4, 9, 16]


See how we started with the empty list, but then `append`ed each squared value of the loop variable `k` as the loop executed. At the end we had a list of the squared `k` values. You can see how a loop with lists can help us to keep a scratchpad with the accumulated results of a calculation, like the solution of a differential equation!


## Arrays

An array is much like a list with three important differences:

1. Homogeneity: An array has to have values of all the same *type* (e.g., integer, float, object, etc.)
2. Fixed size: While it's technically possible to adjust an array's size, it's tricky. We don't generally do that.
2. Mathability: When you do math with arrays it *just* *works*

There are some cases where it doesn't matter much if you use a list or an array (e.g., `plt.plot()`). However here are two basic guidelines:

1. If you need to `append` things to it, use a list.
2. If you need to do math with it, use an array.


# The Euler method <a name="ref1"></a> [[1]](#note1)

Imagine you have a system described by a "state" **s**. The idea is that **s** is the collection of all the variables that are needed to specify the system's condition at some moment in time. If you know **s** at some time, then you know all you need to know to understand what the system is doing at that time. The idea is that there is some rule that determines the rate of change of **s** that depends on **s**, and possibly also on the time like so:

$$\frac{ds}{dt} = f_s(s,t)$$
    
So.. if we have a rule like this, and we know the state of the system now, Euler's method allows us to estimate the state at a later time. How? Simple.. $f_s(s,t)$ is telling us the rate of change of **s** itself. So, if a short period of time $\Delta t$ passes what should be the corresponding change in s (called $\Delta s$)?

$$\frac{\Delta s}{\Delta t} \approx \frac{ds}{dt} = f_s(s,t)$$

Solving for $\Delta{s}$ gives:

$$\Delta s \approx f_s(s,t) \Delta t$$

That's the *Euler Method*! The only thing left is to add $\Delta s$ to the old value of **s**, and repeat the process many times until you get the value of s at some later time.

$$s_{\rm new} = s_{\rm old} + \Delta s$$

# Example
    
Let's work out a detailed example 'by hand'. Conside the first example from the last activity:

$$ \frac{dy}{dx} = y' = - y/5 $$

In this case our "state" is just a number, $y$. So we can rewrite this using the state concept as:

$$\frac{ds}{dt} = f_s(s,t) = -s/5$$

<a name="note1"></a> [^](#ref1) 1. Much of this was taken from my Scientific Computing lesson on the Euler Method: [Euler Method](https://github.com/sspickle/sci-comp-notebooks/blob/master/P01-Euler.ipynb)
